In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../")

In [2]:
import torch

from xent.tasks import Closure
from xent.models import M
from xent.lang import X
from xent.dataprocessing import Wikipedia
from xent.config import *

In [3]:
model = M("gpt2", "M0", base="base")
checker_model = M("gpt2", "M1-zero", base="closure")

In [4]:
corpus_generator = Wikipedia(split=0.8)
get_test_sample = corpus_generator.get_random_test_text

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [10]:
task = Closure(model)
synth = task.generate(get_test_sample, space="tokens")

In [6]:
cut = task.find_xstring(synth[0], X.xreturn)
tasktokens = model.tokenize(synth[0]).input_ids.squeeze(0)

In [7]:
cut = cut + 5

toks = tasktokens[cut::]
check = model.detokenize(toks, mode="tensor")

model.tokenize(".:")

# .: is out there seeking to destroy your whole career as a computer scientist indeed 
# it will break the slicing of your tensor and it will probably also steal your car

{'input_ids': tensor([[11207]], device='cuda:2'), 'attention_mask': tensor([[1]], device='cuda:2')}

In [8]:
numbers = [" 0"," 1"," 2"," 3"," 4"," 5"," 6"," 7"," 8"," 9"," 10"," 11"," 12"," 13"," 14"," 15"," 16"," 17"," 18"," 19"," 20"]
numtoks = torch.tensor([model.tokenize(num).input_ids for num in numbers]).to(device)

In [9]:
# Check if each element in toks is in numtoks
is_number = torch.tensor([tok in numtoks for tok in toks])
# Get indices where is_number is True
number_positions = torch.where(is_number)[0]
print(model.detokenize(toks[number_positions], mode="tensor"))

 9 1 9 11 0 0 0 14 4 3 4 5 3 4 5 0 11 3 3 1 5 7 7 3 4 0 3 5 2 1 4 0 1 8 7 5 2 3 9 3 0 1 0 1 1 0 6 3 2 16 3 4 1 4 19 3 4 3 2 4 2 0 2 5 4 3 4 4 0 1 4 2 1 9 10 3 8 2 6 4 6 2 6 4 4 1 5 0 2 7 3 4 5 0 1 4 4 10 1 9 1 10 4 0 4 6 1 0 3 0 8 2 1 2 1 2 5 0 1 1 6 4 8 2 3 10 2 7 6 7 1 3 4 0 4 1 0 2 9 8 2 1 7 8 1 0 5 3 4 0 1 0 2 5 2 2 7 0 0 1 19 2 5 4 7 4 4 20 3 1 2 1 2 0 0 7 1 2 0 6 1 1 0 6 2 0 0 1 8 2 4 3 4 11 1 3 6 7 0 3 4 7 5 11 1 0


In [10]:
tasktokens = tasktokens.unsqueeze(0)

In [11]:
import torch.nn.functional as F

checker_model.model.eval()
with torch.no_grad():
    logits = checker_model.model(tasktokens).logits
    loss = F.cross_entropy(logits[0, cut:-1], tasktokens[0, cut+1:], reduction="none")
    # Get the predicted token probabilities
    probs = F.softmax(logits[0, cut:-1], dim=-1)
    # Get the indices of tokens with highest probabilities
    highest_prob_tokens = torch.argmax(probs, dim=-1)
    # Convert to list for easier inspection

In [20]:
cut_toks = toks[1:]
pred_toks = highest_prob_tokens
loss = loss

print(cut_toks.shape, pred_toks.shape, loss.shape)

is_number = torch.tensor([tok in numtoks for tok in cut_toks])
numpos = torch.where(is_number)[0]

# for origin, pred, l in zip(cut_toks[numpos], pred_toks[numpos], loss[numpos]):
#     print(f"original: {model.detokenize(origin, mode='tensor')} \t| predicted: {model.detokenize(pred, mode='tensor')} \t| loss: {l}")

torch.Size([801]) torch.Size([801]) torch.Size([801])


tensor([ 513,  860,  657,  860,  513,  657,  657,  657,  657,  657,  657,  657,
         604,  718,  604,  362,  657,  718,  362,  604,  352,  642,  767,  718,
         513,  604,  657,  604,  604,  352,  352,  604,  657,  362,  807,  352,
         604,  352,  362,  807,  657,  657,  352,  657,  352,  352,  657,  718,
         642,  362, 1467,  513,  513,  352,  678,  604,  513,  352,  362,  362,
         362,  352,  352,  362,  604,  604,  352,  362,  513,  352,  362,  362,
         362,  362,  807,  362,  362,  718,  352,  604,  362,  604,  352,  604,
         352,  604,  352,  362,  657,  362,  513,  362,  604,  657,  657,  352,
         807,  362,  604,  362,  807,  352,  807,  657,  352,  604,  642,  352,
         657,  513,  362,  718,  352,  362,  352,  362,  513,  642,  657,  352,
         362,  362,  604,  604,  352,  362,  807,  352,  718,  807,  718,  657,
         807,  642,  657,  604,  718,  657,  352,  807,  718,  657,  352,  767,
         604,  657,  657,  604,  362,  6

## Let's try another approach. 

NameError: name 'cut_toks' is not defined